<h1>STA 141B Lab 1 </h1><br>
<font size = '4px'>In this lab, I will add comments to a python script I wrote to load data into a PostgreSQL database using a predefined database schema. </font>

    import psycopg2
    import os
    import sys
    import csv
    from subprocess import check_output

Libraries used and motivation:
<ul>
<li>The psycopg2 library is used to interact with PostgreSQL from python.</li>
<li>The os library is used to capture the environment variable.</li>
<li>The sys library is used to end to program upon error. </li>
<li>The csv library is used to read in the csv files. </li>
<li>The check_output function from subprocess library gets output returned by BASH commands. 
</ul>

    nhtsfiles = ['HHV2PUB.CSV','PERV2PUB.CSV','VEHV2PUB.CSV', 'DAYV2PUB.CSV']
    #All the variables with type int in codebook
    nhtsintvars = ['DRVRCNT', 'HHSIZE', 'HHVEHCNT', 'NUMADLT', 'RESP_CNT', 'VARSTRAT', 'WRKCOUNT', 
				'WTHHFIN', 'CARRODE', 'CNTTDTR', 'DELIVER', 'DISTOWK', 'FMSCSIZE', 
				'GCDWORK', 'HHVEHCT', 'LSTTRDAY', 'MCUSED', 'NBIKETRP', 
				'NWALKTRP', 'PTUSED', 'PURCHASE', 'R_AGE', 'SFWGT', 'TIMETOWK', 'TOSCSIZE', 
				'VARSTRAT', 'WKFMHMXX', 'WTPERFIN', 'YEARMILE', 'YRTOUS', 
				'ANNMILE', 'BESTMILE', 'EIADMPG', 'EPATMPG', 'FUELTYPE', 
				'GSCOST', 'GSTOTCST', 'GSYRGAL', 
				'OD_READ', 'VEHAGE', 'VEHOWNMO', 'VEHYEAR', 'WTHHFIN', 
				'DWELTIME', 'GASPRICE', 'HH_ONTD', 'NONHHCNT', 
				'NUMONTRP', 'R_AGE', 'TRACCTM', 'TREGRTM', 'TRPACCMP', 'TRPHHACC', 'TRPMILES',
				'TRVLCMIN', 'TRVL_MIN', 'TRWAITTM', 'VMT_MILE', 'WTTRDFIN']
    nhtsfilenames = ['household', 'person', 'vehicle', 'travelday']
    eiafiles = ['EIA_CO2_Electricity_2015.csv','EIA_CO2_Transportation_2015.csv','EIA_MkWh_2015.csv']
    eiafilenames = ['electricity', 'transportation', 'mkwh']

Create arrays denoting all csv files from the nhts csv and eia csv as well as their variables, so we can loop over these to populate the rows of the database later on. 

    def wc(filename):
        return int(check_output(["wc", "-l", filename]).split()[0])

    def getCreateCmd_nhts(tablename, header): #Program to get the create table command
        allvars = []
        append = allvars.append
        for var in header:
            if(var in nhtsintvars):
                append(var + ' ' +  'integer')
            else:
                append(var + ' ' + 'varchar(20)')
        createcmd = 'CREATE TABLE nhts.%s (%s);' % (tablename, ",".join(allvars))
        return createcmd

    def getInsertCmd_nhts(header):
        string = []
        append = string.append
        for i in range(0,len(header)):
            if(header[i] in nhtsintvars):
                append("float(row[%d])" % i)
            else:
                append("row[%d]" % i)
        insertcmd = "%s" % (','.join(string))
        return insertcmd

Functions defined:
<ul>
<li><b>wc</b> gets the line count of the file.</li>
<li><b>getCreatedCmd_nhts</b> generates the SQL command to create the nhts table.</li> 
<li><b>getInsertCmd_nhts</b> generates the SQL command to insert the columns. 
</ul>


    con = None

    try:
        con = psycopg2.connect(database = "postgres", user =  os.environ['USER'] )
        cur = con.cursor()
        cur.execute ('CREATE SCHEMA IF NOT EXISTS NHTS;')

        #READ IN NHTS DATA
        for i in range(0, len(nhtsfiles)):
            csvfile = open('./ecs165a/%s' % nhtsfiles[i],'rb')
            reader = csv.reader(csvfile)
            header = next(reader)
            createcmd = getCreateCmd_nhts(nhtsfilenames[i],header)
            cur.execute(createcmd)
            insertcmdformat = getInsertCmd_nhts(header)
            filewc = wc('./ecs165a/%s' % nhtsfiles[i])
            rowCount = 0 
            TotalCount = 0 #Total rows read
            stringroup = []
            for row in reader:
                t = eval(insertcmdformat)
                stringroup.append(t)
                rowCount += 1
                TotalCount += 1
                if(rowCount == 50000):
                    insertcmd = "INSERT INTO nhts.%s VALUES %s;" % (nhtsfilenames[i], str(stringroup).strip('[]'))
                    cur.execute(insertcmd)
                    rowCount = 0
                    stringroup = []

                elif ((filewc - 1 == TotalCount) and (filewc - TotalCount - 1) < 50000): 
                    insertcmd = "INSERT INTO nhts.%s VALUES %s;" % (nhtsfilenames[i], str(stringroup).strip('[]'))
                    cur.execute(insertcmd)

            print("NHTS file " + str(i) + " Done" + "TotalCount" + str(TotalCount))

        #READ IN EIA DATA
        cur.execute('CREATE SCHEMA IF NOT EXISTS EIA;')
        for i in range(0, len(eiafiles)):
            csvfile = open('./ecs165a/%s' % eiafiles[i],'rb')		
            reader = csv.reader(csvfile)
            header = next(reader)
            createcmd = 'CREATE TABLE eia.%s (%s);' % (eiafilenames[i], 'MSN varchar(100), yyyymm varchar(100), Values varchar(100)')
            cur.execute(createcmd)
            #insertcmdformat = getInsertCmd_eia(header)
            filewc = wc('./ecs165a/%s' % eiafiles[i])
            rowCount = 0
            TotalCount = 0
            stringroup = []
            for row in reader:
                t = eval('(row[0], row[1], row[2])')
                stringroup.append(t)
                rowCount += 1
                TotalCount += 1
                if(rowCount == 50000):
                    insertcmd = "INSERT INTO eia.%s VALUES %s;" % (eiafilenames[i], str(stringroup).strip('[]'))
                    cur.execute(insertcmd)
                    rowCount = 0
                    stringroup = []
                elif ((TotalCount == filewc - 1) and (filewc - TotalCount - 1) < 50000): 
                    insertcmd = "INSERT INTO eia.%s VALUES %s;" % (eiafilenames[i], str(stringroup).strip('[]'))
                    cur.execute(insertcmd)
            print("EIA file " + str(i) + " Done" + "TotalCount" + str(TotalCount))

        cur.execute('ALTER TABLE nhts.household ADD PRIMARY KEY (houseid);')
        cur.execute('ALTER TABLE nhts.person ADD PRIMARY KEY (houseid, personid);')
        cur.execute('ALTER TABLE nhts.vehicle ADD PRIMARY KEY (houseid, vehid);')
        cur.execute('ALTER TABLE nhts.travelday ADD PRIMARY KEY (houseid, personid, tdtrpnum);')
        cur.execute('ALTER TABLE eia.transportation ADD PRIMARY KEY (msn, yyyymm);')
        cur.execute('ALTER TABLE eia.electricity ADD PRIMARY KEY (msn, yyyymm);')
        cur.execute('ALTER TABLE eia.mkwh ADD PRIMARY KEY (msn, yyyymm);')

        con.commit()

    except psycopg2.DatabaseError, e:
        if con:
            con.rollback()
        print 'Error %s' % e    
        sys.exit(1)


    finally:    
     if con:
        con.close()


First we set connection to none. Then we try to connect to the postgresql database and set a cursor to that connection. If schema for nhts is not define, we will define it. 
<br>
In the first for loop, we looped through all the nhts file in the directory. We inserted the rows at 50,000 intervals just to keep track of the progress and when an error occurs, we know the ballpark of the line that caused the problem.
<br>
The second for loop does the same thing as the first for loop but for the eia data files instead. Given that the eia file format follow a fixed structure, we can declare the create table and insert command in the loop instead of making functions to create different commands for different files. 
<br>
The last couple lines of identify the primary keys of each data table. 